In [1]:
from btgp.data_utils import load_dataset
from btgp.gp import ONGP
import torch
device = torch.device("cuda:4" if torch.cuda.is_available() else "cpu")


In [2]:
path_start = "../data/uci/"

train_x, train_y, val_x, val_y, test_x, test_y = load_dataset(path_start + "houseelectric/houseelectric.mat")

dataset_path: ../data/uci/houseelectric/houseelectric.mat
data.shape: torch.Size([2049280, 12])
train_x.shape: torch.Size([1311539, 11])
full kernel size: 6408.0 GB
train_x: min 0.000, max 1.000, mean 0.325, std 0.319
val_x: min 0.000, max 1.000, mean 0.325, std 0.319
test_x: min -0.009, max 1.000, mean 0.325, std 0.319
train_y: min -2.300, max 2.867, mean -0.000, std 1.000
val_y: min -2.300, max 2.772, mean -0.000, std 0.999
test_y: min -2.300, max 2.824, mean -0.001, std 0.999



In [3]:
torch.manual_seed(0)
print("A single GP, no weight optimization (uniform weights):")
test_y = test_y.to(device=device)
gp = ONGP(train_x.to(dtype=torch.float64, device=device),
          train_y.to(dtype=torch.float64, device=device),
          test_x.to(dtype=torch.float64, device=device), lambd=1e-4, precision=8)
gp.random_bit_order()
gp.process()
print("Train nll:", gp.nll)
print("Test nll:", gp.calculate_test_nll(test_y))
print("Test rmse:", gp.rmse(test_y))
print()
print("Training the weight vector and the bit order:")
for _ in range(2):
    gp.update_weights(update_order=True, verbose=True)
    gp.save()
    print("Test nll:", gp.calculate_test_nll(test_y))
    print("Test rmse:", gp.rmse(test_y))

A single GP, no weight optimization (uniform weights):


/home/sdaulton/anaconda3/envs/effgp/lib/python3.9/site-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


Train nll: tensor(0.7376, device='cuda:4', dtype=torch.float64)
Test nll: tensor(0.6646, device='cuda:4', dtype=torch.float64)
Test rmse: tensor(0.2205, device='cuda:4', dtype=torch.float64)

Training the weight vector and the bit order:
train nll: tensor(0.4764, device='cuda:4', dtype=torch.float64)
train nll: tensor(6.4472, device='cuda:4', dtype=torch.float64)
nll increased; revert
train nll: tensor(0.4764, device='cuda:4', dtype=torch.float64)
bit order: tensor([ 7,  1, 10,  3,  7,  2,  2,  1,  8,  4,  9,  3, 10,  4,  0,  1,  9,  2,
         5, 10,  6,  8,  4,  9, 10,  9,  9,  0,  5,  7, 10,  7,  7,  5,  6,  6,
         0,  3,  1, 10,  3,  9,  6,  8,  4,  1,  7,  8,  3,  3,  6,  2,  0,  6,
         5,  2,  8,  6,  9,  8,  2,  6,  0,  5,  4, 10,  1, 10,  9,  5,  4,  0,
         1,  4,  3,  0,  2,  4,  7,  5,  8,  8,  0,  7,  5,  1,  3,  2],
       device='cuda:4')
Test nll: tensor(0.3577, device='cuda:4', dtype=torch.float64)
Test rmse: tensor(0.1818, device='cuda:4', dtype=torch.fl

## test SLSQP

In [4]:
torch.manual_seed(0)
print("A single GP, no weight optimization (uniform weights):")
test_y = test_y.to(device=device)
gp = ONGP(train_x.to(dtype=torch.float64, device=device),
          train_y.to(dtype=torch.float64, device=device),
          test_x.to(dtype=torch.float64, device=device), lambd=1e-4, precision=8)
gp.random_bit_order()
gp.process()
print("Train nll:", gp.nll)
print("Test nll:", gp.calculate_test_nll(test_y))
print("Test rmse:", gp.rmse(test_y))
print()
print("Training the weight vector and the bit order:")
for _ in range(2):
    gp.update_weights_slsqp(update_order=True, verbose=True)
    gp.save()
    print("Test nll:", gp.calculate_test_nll(test_y))
    print("Test rmse:", gp.rmse(test_y))

A single GP, no weight optimization (uniform weights):
Train nll: tensor(0.7376, device='cuda:4', dtype=torch.float64)
Test nll: tensor(0.6646, device='cuda:4', dtype=torch.float64)
Test rmse: tensor(0.2205, device='cuda:4', dtype=torch.float64)

Training the weight vector and the bit order:
Fitting failed with the optimizer reporting 'Iteration limit reached'
bit order: tensor([ 7,  7,  1, 10,  3,  0,  2,  2,  1,  8,  4,  9,  3, 10,  4,  9,  1,  2,
         8,  5, 10,  6,  7,  4,  9, 10,  9,  9,  0,  5,  7, 10,  7,  5,  1,  6,
         6,  0,  3, 10,  3,  7,  9,  6,  8,  4,  1,  3,  8,  3,  5,  6,  2,  6,
         2,  5,  8,  6,  9,  8,  2,  6,  0,  5,  4, 10,  1, 10,  9,  0,  4,  1,
         4,  3,  0,  2,  4,  7,  5,  8,  8,  0,  7,  5,  1,  3,  2,  0],
       device='cuda:4')
Test nll: tensor(-0.5934, device='cuda:4', dtype=torch.float64)
Test rmse: tensor(0.1519, device='cuda:4', dtype=torch.float64)
bit order: tensor([ 7,  7,  1, 10,  3,  0,  1,  2,  2,  8,  3,  4,  9,  5, 10,  4